<a href="https://colab.research.google.com/github/aranpandey/CornyWebbedRegisters/blob/master/Temp%2BMortality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')
import DLYHandler
import pandas as pd
import numpy as np
import scipy.stats as st
from datetime import datetime
from DLYHandler import read_ghcn_data_file

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
#TBD Reassess this method to see if we only take upper bound for TMX and lower bound fro TMIN. Yes choosing to do this now
def classifyIfInRange(row):
    #if row['TMAX'] < row['ci_low']:
     #   return 1
    #elif row['TMAX'] >row['ci_high']:
    if row['TMAX'] >row['ci_high']:
        return 1
    else:
     return 0


def mean_confidence_interval(df_small, confidence=0.95):
    m=df_small['TMAX'].mean()

    se=st.sem(df_small['TMAX'] )
    
    h=st.norm.interval(0.95,m,se)
    
    return m, h

#This function calculates the mean and confidence interval for each week of the year. That is for 52 weeks.
#Need to do this for each year. TBD- adust the function to do for each year.
def populate_weekly_confidence(base_data,all_data):
    #loop through each record to find mean confidence interval for each row
    for i in range(1, 53):
       df_week= base_data [(base_data['Week'] ==i)] 
       upper_alt=mean_confidence_interval(df_week)    
       all_data.loc[all_data['Week'] == i, 'ci_low'] = upper_alt[1][0]
       all_data.loc[all_data['Week'] == i, 'ci_high'] = upper_alt[1][1]
    

In [ ]:
df_temp_stn=read_ghcn_data_file(filename="/content/gdrive/My Drive/Colab Notebooks/USC00011084.dly",
                        variables=['TMAX', 'TMIN'], include_flags=False,
                        dropna='all')


df_temp_stn.reset_index(inplace=True)
df_temp_stn= df_temp_stn.rename(columns = {'index':'OBSDT'})
df_temp_stn.head(1000) 

df_temp_stn.dtypes


#Divide tmax and tmin by 10 as the unit is 'tenths of degrees C'
df_temp_stn['TMAX']= df_temp_stn['TMAX']/10
df_temp_stn['TMIN']= df_temp_stn['TMIN']/10

#degree farenheight

df_temp_stn['TMAX']= (df_temp_stn['TMAX']*1.8)+32
df_temp_stn['TMIN']= (df_temp_stn['TMIN']*1.8)+32


#add 'week of year' to the data
df_temp_stn["Week"] =df_temp_stn.OBSDT.apply(lambda x:x.weekofyear)
df_temp_stn["Year"] =df_temp_stn.OBSDT.apply(lambda x:x.year)

df_temp_stn.head(10)


/content/gdrive/My Drive/Colab Notebooks/DLYHandler.py:128: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace(-9999.0, pd.np.nan, inplace=True)


ELEMENT,OBSDT,TMAX,TMIN,Week,Year
0,1926-01-21,75.02,51.98,3,1926
1,1926-01-22,71.96,37.04,3,1926
2,1926-01-23,41.00,28.94,3,1926
3,1926-01-24,41.00,33.98,3,1926
4,1926-01-25,57.92,35.06,4,1926
5,1926-01-26,57.92,35.96,4,1926
6,1926-01-27,60.98,37.04,4,1926
7,1926-01-28,57.92,39.02,4,1926
8,1926-01-29,59.00,42.98,4,1926
9,1926-01-30,55.04,41.00,4,1926


In [ ]:
'''Get the  dly file ready'''

import datetime

#get the yearly, monthly, weekly mean ( Use baseline years as 1836 to 1970).
#The baseline period is used to measure mean and confidence interval

date1 = pd.to_datetime(datetime.date(1935,12,31))
date2 = pd.to_datetime(datetime.date(1970,12,31))
#df_base=    df_temp_stn.loc[(df_temp_stn['OBSDT'] >=date1) & (df_temp_stn['OBSDT'] <=date2)]
df_base= df_temp_stn 

#df_base.head(10)

year_max_mean=df_base.groupby(df_base.OBSDT.dt.year)['TMAX'].mean().reset_index()
month_max_mean=df_base.groupby(df_base.OBSDT.dt.month)['TMAX'].mean().reset_index()
week_max_mean=df_base.groupby(df_base.OBSDT.dt.week)['TMAX'].mean().reset_index()
daily_max_mean=df_base.groupby(df_base.OBSDT.dt.day)['TMAX'].mean().reset_index()
daily_min_mean=df_base.groupby(df_base.OBSDT.dt.day)['TMIN'].mean().reset_index()
week_min_mean=df_base.groupby(df_base.OBSDT.dt.week)['TMIN'].mean().reset_index()



print(year_max_mean)
print(month_max_mean)
print(week_max_mean)

In [ ]:
#All data ( not including baseline)
#df_alldata=    df_temp_stn.loc[(df_temp_stn['OBSDT'] >date2)]
df_alldata=df_temp_stn 

#df_alldata["Mean"] = np.where(df_alldata["Week"] ==2.5, "False", "True")

# Get individual records at weekly level to count the number of days outside of confidence interval
#week_grain=df_alldata.groupby(['Year','Week'])['TMAX'].mean().reset_index()
#week_grain.head(10)

df_base1=df_base.groupby(['Year','Week'])['TMAX'].mean().reset_index()
#print(df_base1)
#bare minimum base data for statistical math
df_base_small=pd.DataFrame()
df_base_small['Week']=df_base1['Week']
df_base_small['TMAX']=df_base1['TMAX']

#print(df_base_small.head(20))

df_dataall_week=df_alldata.groupby(['Year','Week'])['TMAX'].mean().reset_index()

#populate ci_low, ci_high for each record ( each daily record - do we have 
###confidence that temp of this day of week is within 95% range for this week's temperature) 
populate_weekly_confidence( df_base_small, df_dataall_week)
#now populate  if ci  in range
df_dataall_week["inRange"] = df_dataall_week.apply(lambda row: classifyIfInRange(row), axis=1)
df_dataall_week.head(10)

,Year,Week,TMAX,ci_low,ci_high,inRange
0,1926,3,57.245000,61.312198,64.616107,0
1,1926,4,59.540000,60.981584,64.219992,0
2,1926,5,66.148571,62.471554,65.507634,1
3,1926,6,68.000000,62.862619,65.555757,1
4,1926,7,69.234286,65.186040,67.799642,1
5,1926,8,71.291429,66.611418,69.248907,1
6,1926,9,62.728571,66.727578,69.310117,0
7,1926,10,60.002857,69.004623,71.543221,0
8,1926,11,72.962857,71.911274,74.406778,0
9,1926,12,70.725714,72.969212,74.997685,0


In [ ]:
df_w = df_dataall_week.groupby('Year')['inRange'].sum().reset_index()

df_w.head

In [ ]:
#read mortality data
df_mor = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Deaths_1x1.csv')

df_mor.head()

df_mor.dtypes
df_mor.columns = df_mor.columns.str.replace(' ', '')
df_mor_year = df_mor.groupby(['Year'])['Total'].sum().reset_index()
print(df_mor['Year'].agg(['min', 'max']))

df_mor_year.tail(500)
len(df_mor)
df_mor.head(1000)
df_mor['Year'] = df_mor['Year'].str.strip()
df_mor['Year'] = df_mor['Year'].str[:4]
df_mor_year = df_mor.groupby(['Year'])['Total'].sum().reset_index()

df_mor_year.head(1000)

In [ ]:
df_dataall_week.dtypes
df_mor_year['Year'] = df_mor_year['Year'].astype('int64')

df_dataall_week['Year'] = df_dataall_week['Year'].astype('int64')
df_w.astype('int64')
df_merged= pd.merge(df_w, df_mor_year, on="Year").reset_index() 
df_merged.head(100)

In [ ]:
inputs = df_merged.to_numpy()             
outputs = df_merged.to_numpy()
from matplotlib import pyplot as plt
split = int(0.8*len(df_merged))
X_train, X_test, y_train, y_test = inputs[:split], inputs[split:], outputs[:split], outputs[split:]
X_train = np.expand_dims(X_train,axis=2)  
X_test = np.expand_dims(X_test,axis=2) 

print(split)


65


82

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor
model = RidgeCV(cv = 2)
X_train, X_test, y_train, y_test = inputs[:split], inputs[split:], outputs[:split], outputs[split:]
model.fit(X_train, y_train)
print("{}".format(np.linalg.norm(model.predict(X_test) - y_test, 1)/len(y_test)))
predict = model.predict(X_test)

print(predict)
print(predict.shape)

cross_val_score(model, X_train, y_train, cv = 2)

0.0003491289589076943
[[6.50000739e+01 1.99800007e+03 2.49995335e+01 2.33725595e+06]
 [6.60001952e+01 1.99900020e+03 2.09997283e+01 2.39139906e+06]
 [6.70001827e+01 2.00000018e+03 2.49995079e+01 2.40335101e+06]
 [6.80001680e+01 2.00100017e+03 1.40000979e+01 2.41642491e+06]
 [6.90002023e+01 2.00200020e+03 1.30001417e+01 2.44338691e+06]
 [7.00001645e+01 2.00300016e+03 1.10002486e+01 2.44828803e+06]
 [7.09999475e+01 2.00399995e+03 1.20002161e+01 2.39761502e+06]
 [7.20000649e+01 2.00500006e+03 2.89992773e+01 2.44801799e+06]
 [7.29999426e+01 2.00599994e+03 3.19991248e+01 2.42626395e+06]
 [7.39998779e+01 2.00699988e+03 2.29996132e+01 2.42371199e+06]
 [7.49999779e+01 2.00799998e+03 1.30001347e+01 2.47198384e+06]
 [7.59998111e+01 2.00899981e+03 1.10002578e+01 2.43716302e+06]
 [7.69998645e+01 2.00999986e+03 2.29995972e+01 2.46843489e+06]
 [7.79999651e+01 2.01099997e+03 2.49994705e+01 2.51545786e+06]
 [7.90000018e+01 2.01200000e+03 2.29995680e+01 2.54327895e+06]
 [8.00001218e+01 2.01300012e+03 1

array([0.99999996, 0.99999999])

In [ ]:
model.score(X_test, y_test)

0.9999999986409805